In [1]:
import DSSATTools
from DSSATTools import Weather, SoilProfile, soil
import pandas as pd
import numpy as np

In [2]:
DATES = pd.date_range('2000-01-01', '2010-12-31'); N=len(DATES)
df = pd.DataFrame(
        {
        'tn': np.random.gamma(10, 1, N),
        'rad': np.random.gamma(10, 1.5, N),
        'prec': np.round(np.random.gamma(.4, 10, N), 1),
        'rh': 100 * np.random.beta(1.5, 1.15, N),
        },
        index=DATES,
    )
df['TMAX'] = df.tn + np.random.gamma(5., .5, N)

weather = Weather(
    df, 
    {"tn": "TMIN", "rad": "SRAD", "prec": "RAIN", 
    "rh": "RHUM", "TMAX": "TMAX"},
    4.54, -75.1, 1800, 
)
weather.description = 'Manhattan Kansas'
weather.INSI = 'MHKS'
weather.WNDHT = 2

first_year = df.index[0].year
total_years = df.index[-1].year - df.index[0].year + 1
weather._name = f'{weather.INSI}{str(first_year)[2:]}{total_years:02d}'
weather.write('../DSSAT/')

/opt/anaconda3/lib/python3.12/site-packages/DSSATTools/weather.py:215: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = self.data.applymap(lambda x: f"{x:5.1f}")


In [79]:
soilGrid = pd.read_csv('../Row data for model/soilGridArmennia.csv', index_col=0)
soilGrid['SLB'] = soilGrid['Depth'].str.split("-", expand=True)[1]
soilGrid['SLB'] = pd.to_numeric(soilGrid['SLB'], errors='coerce').astype('int')

soilGrid.rename(columns={'location':'location', 'lat':'lat', 'lon':'lon',
                        'SALB':'SALB', 'CN2':'SLRO', 'soilType':'soilType', 'Depth':'Depth', 'Thickness':'Thickness',
                        'ParticleSizeClay':'Clay', 'ParticleSizeSilt':'Silt', 'ParticleSizeSand':'Sand', 
                        'BD':'BD', 'LL15':'LL15', 'DUl':'DUL', 'SAT':'SAT', 'PH':'PH', 'Wheat.XF':'SRGF', 'SLB':'SLB'}, inplace=True)

soilInf = {'Gyumri':{'site': 'Gyumri    ',
                     'id': 'KSUGY24001',
                    'csc_family': 'Chernozem',
                    'SLU1': 10.2,
                    'SLPF': 0.8},
           'Masrik':{'site': 'Masrik    ',
                     'id':'KSUMA24001',
                    'csc_family': 'csc2',
                    'SLU1': 10.2,
                    'SLPF': 0.6},
           'Stepanavan':{'site': 'Stepanava',
                         'id':'KSUST24001',
                    'csc_family': 'csc1',
                    'SLU1': 10.2,
                    'SLPF': 0.7},
          'Sisan':{'site': 'Sisan     ',
                   'id':'KSUSI24001',
                    'csc_family': 'csc3',
                    'SLU1': 10.2,
                    'SLPF': 0.9},
          
          }
layerDepth = [5, 15, 30, 60, 100, 200]

all_soil_types = []

for i in soilInf.keys():
    Location = soilGrid.loc[soilGrid['location']==i]
    soilprofile = SoilProfile(
    pars={'site': soilInf[i]['site'],
         'country': 'Armenia',
          'description': Location['soilType'].unique()[0][11:],
         'lat': Location['lat'].unique()[0],
         'lon': Location['lon'].unique()[0],
         'SALB': Location['SALB'].unique()[0], #Albedo
         'id': soilInf[i]['id'],
         'SLU1': soilInf[i]['SLU1'], # Stage 1 Evaporation (mm)
         'SLPF': soilInf[i]['SLPF'], # Soil fertility factor
         'SLRO': Location['SLRO'].unique()[0],
         'csc_family': soilInf[i]['csc_family']
    })
    layer1 = Location.loc[(Location['SLB']== layerDepth[0])]
    layer2 = Location.loc[(Location['SLB']== layerDepth[1])]
    layer3 = Location.loc[(Location['SLB']== layerDepth[2])]
    layer4 = Location.loc[(Location['SLB']== layerDepth[3])]
    layer5 = Location.loc[(Location['SLB']== layerDepth[4])]
    layer6 = Location.loc[(Location['SLB']== layerDepth[5])]

    layers = [
    soil.SoilLayer(layer1['SLB'].iloc[0], {'SLLL': layer1['LL15'].iloc[0], 'SDUL':layer1['DUL'].iloc[0], 'SSAT':layer1['SAT'].iloc[0], 'SLHW':layer1['PH'].iloc[0], 'SLCL': layer1['Clay'].iloc[0], 'SLSI': layer1['Silt'].iloc[0], 'SRGF': layer1['SRGF'].iloc[0], 'SLB': layer1['BD'].iloc[0]}),
    soil.SoilLayer(layer2['SLB'].iloc[0], {'SLLL': layer2['LL15'].iloc[0], 'SDUL':layer2['DUL'].iloc[0], 'SSAT':layer2['SAT'].iloc[0], 'SLHW':layer2['PH'].iloc[0], 'SLCL': layer2['Clay'].iloc[0], 'SLSI': layer2['Silt'].iloc[0], 'SRGF': layer2['SRGF'].iloc[0], 'SLB': layer2['BD'].iloc[0]}),
    soil.SoilLayer(layer3['SLB'].iloc[0], {'SLLL': layer3['LL15'].iloc[0], 'SDUL':layer3['DUL'].iloc[0], 'SSAT':layer3['SAT'].iloc[0], 'SLHW':layer3['PH'].iloc[0], 'SLCL': layer3['Clay'].iloc[0], 'SLSI': layer3['Silt'].iloc[0], 'SRGF': layer3['SRGF'].iloc[0], 'SLB': layer3['BD'].iloc[0]}),
    soil.SoilLayer(layer4['SLB'].iloc[0], {'SLLL': layer4['LL15'].iloc[0], 'SDUL':layer4['DUL'].iloc[0], 'SSAT':layer4['SAT'].iloc[0], 'SLHW':layer4['PH'].iloc[0], 'SLCL': layer4['Clay'].iloc[0], 'SLSI': layer4['Silt'].iloc[0], 'SRGF': layer4['SRGF'].iloc[0], 'SLB': layer4['BD'].iloc[0]}),
    soil.SoilLayer(layer5['SLB'].iloc[0], {'SLLL': layer5['LL15'].iloc[0], 'SDUL':layer5['DUL'].iloc[0], 'SSAT':layer5['SAT'].iloc[0], 'SLHW':layer5['PH'].iloc[0], 'SLCL': layer5['Clay'].iloc[0], 'SLSI': layer5['Silt'].iloc[0], 'SRGF': layer5['SRGF'].iloc[0], 'SLB': layer5['BD'].iloc[0]}),
    soil.SoilLayer(layer6['SLB'].iloc[0], {'SLLL': layer6['LL15'].iloc[0], 'SDUL':layer6['DUL'].iloc[0], 'SSAT':layer6['SAT'].iloc[0], 'SLHW':layer6['PH'].iloc[0], 'SLCL': layer6['Clay'].iloc[0], 'SLSI': layer6['Silt'].iloc[0], 'SRGF': layer6['SRGF'].iloc[0], 'SLB': layer6['BD'].iloc[0]})
    ]                                          
    for layer in layers: 
        soilprofile.add_layer(layer)
    all_soil_types.append(soilprofile)

file = open("../DSSAT/SOL.SOL", "w")
file.write('*SOILS: General DSSAT Soil Input File \n')

for i in all_soil_types:
    file.write(f'{i}\n')
file.close()